In [2]:
# Setup Enviroment

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# cd drive/My Drive/Thesis/

In [5]:
#%run thesis_gpars.ipynb

In [6]:
import time
import re
import math
import queue
import random
import json
import multiprocessing
import os

# DATA STRUCTURE 
Resp thesis_gpars.ipynb


In [7]:
MODIFY_INDEX = 0
VACANT_INDEX = -1
VACANT_LABEL = -1

In [8]:
## Class Edge
class MyEdge(tuple):
  def __repr__(self):
    return 'e {} {} {}'.format(self[0], self[1], self[2])

In [9]:
## Class MyNode

class MyNode:
  def __init__(self, id = VACANT_INDEX, label = VACANT_LABEL, to_nodes = [], from_nodes = []):
    self.id = id
    self.to_nodes = to_nodes.copy()   ## list id_nodes that from MyNode
    self.from_nodes = from_nodes.copy() ## list id_nodes that to MyNode
    self.label = label 

  def __repr__(self):
    return 'v {} {}'.format(self.id, self.label)

In [10]:
## Class MyGraph

class MyGraph:
  def __init__(self, nodes=[], edges=set()):
    self.nodes = nodes.copy()
    self.edges = edges.copy()


  def loadNode (self, filename):  
    count = 0
    with open(filename, 'r') as f:
      for line in f.readlines():
        count += 1
    self.nodes = [None]*count     ## load node id resp index self.nodes 
    with open(filename, 'r') as f:
      for line in f.readlines():
        line = line.strip('\n')
        line = re.split(r'\D+', line)
        self.nodes[int(line[0])+MODIFY_INDEX] = MyNode(int(line[0])+MODIFY_INDEX, int(line[1]))

    
  def loadEdge (self, filename):
    with open(filename, 'r') as f:
      for line in f.readlines():
        line = line.strip('\n')
        line = re.split(r'\D+', line)
        for i in range(0,2):
          line[i] = int(line[i]) + MODIFY_INDEX
        #if len(line) > 2:
         # line[2] = int(line[2])
          #self.edges.append(MyEdge(line[0], line[1], line[2]))
        #else:
        self.edges.add(MyEdge((line[0], line[1], VACANT_LABEL)))   #####
        self.nodes[line[0]].to_nodes.append(line[1])
        self.nodes[line[1]].from_nodes.append(line[0])
  

  def loadData(self, f1, f2):
    self.loadNode(f1)
    self.loadEdge(f2)

  def addNode(self, node):
    self.nodes.append(node)

  def addEdge(self, edge):
    self.edges.add(edge)
    self.nodes[edge[0]].to_nodes.append(edge[1])
    self.nodes[edge[1]].from_nodes.append(edge[0])
  
  def getNumEdges(self):
    return len(self.edges)

  def getDFSLabels(self):
    res = set()
    for e in self.edges:
      vlb1 = self.nodes[e[0]].label
      vlb2 = self.nodes[e[1]].label
      if (vlb1, e[2], vlb2) not in res:
        res.add((vlb1, e[2], vlb2))
    return res

  def getDFSEdge(self, e):
    return MyDFSEdge((e[0], e[1], (self.nodes[e[0]].label, e[2], self.nodes[e[1]].label)))
    
  def getMatchingNodeDict(self, label):  
    ''' func phi - return list of dictionary {0: node_index} they have same label'''
    res = []
    for i in range(len(self.nodes)):
      if self.nodes[i].label == label:
        temp = {0: i}
        res.append(temp)
    return res

  def getMatchingNodeList(self, label):
    res = []
    for i in range(len(self.nodes)):
      if self.nodes[i].label == label:
        res.append([i])
    return res
  
  def getLabelList(self):
    res = []
    for i in self.nodes:
      if i.label not in res:
        res.append(i.label)
    return res

  def getLabelNum(self):
    ''' return dictionary {vlb: num of vlb} '''
    res = dict.fromkeys(self.getLabelList(), 0)
    for i in self.nodes:
      res[i.label] += 1
    return res

  def getLabelDict(self):
    res = dict.fromkeys(self.getLabelList(), None)
    for i in res:
      res[i] = []
    for i in self.nodes:
      res[i.label].append(i.id)
    return res


  def getLabelMatching(self):
    ''' return dictionary {vlb: [vlb to_nodes]}'''
    res = dict.fromkeys(self.getLabelList(), None)
    for i in res:
      res[i] = []
    for i in self.nodes:
      for j in i.to_nodes:
        if self.nodes[j].label not in res[i.label]:
          res[i.label].append(self.nodes[j].label)
    return res


  def getLabelMatchingFrom(self):
    ''' return dictionary {vlb: [vlb from_nodes]}'''
    res = dict.fromkeys(self.getLabelList(), None)
    for i in res:
      res[i] = []
    for i in self.nodes:
      for j in i.from_nodes:
        if self.nodes[j].label not in res[i.label]:
          res[i.label].append(self.nodes[j].label)
    return res
  
  def display(self):
    for n in self.nodes:
      print (n)
    for e in self.edges:
      print (e)
  
  def plot(self):
    try:
      import networkx as nx
      import matplotlib.pyplot as plt
      import matplotlib.colors as mcolors
    except Exception as e:
      print('Can not plot graph: {}'.format(e))
      return
    #cycle = list(mcolors.CSS4_COLORS.keys())
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    gnx = nx.DiGraph()
    vlbs = {}
    elbs = {}
    for i in range(len(self.nodes)):
      gnx.add_node(i, label= self.nodes[i].label)
      vlbs[i] = self.nodes[i].label
    for e in self.edges:
      gnx.add_edge(e[0], e[1], label=e[2])
      elbs[(e[0], e[1])] = e[2]
    
    vcolors = [cycle[i%len(cycle)] for i in vlbs.values()]
    ecolors = 'black'
    #ecolors = [cycle[len(cycle)%i] for i in elbs.values()]
    fsize = (min(12, 1 * len(self.nodes)),
              min(12, 1 * len(self.nodes)))
    plt.figure(3, figsize=fsize)
    # pos = nx.circular_layout(gnx)
    pos = nx.spring_layout(gnx)
    nx.draw_networkx(gnx, pos, node_size = 200, node_color = vcolors, edge_color = ecolors, labels = vlbs)
    #nx.draw_networkx_edge_labels(gnx, pos, edge_labels= elbs)
    plt.autoscale(enable = True)
    plt.show()

  def DFSUtil (self, v, visited, res):
    visited[v] = True
    for i in self.nodes[v].to_nodes:
      if visited[i] == False:
        res.append([v, i])
        self.DFSUtil(i, visited, res)

  def DFS (self, v):
    res = []
    visited = [False] * len (self.nodes)
    self.DFSUtil(v, visited, res)
    return res

  def DFSUtil (self, s, visited, res):  
    stack = []     
    stack.append(s)  
    while (len(stack) != 0): 
      s = stack.pop() 
      if (not visited[s]): 
        # print(s, end = " ")  
        visited[s] = True
      i = 0
      while i < len(self.nodes[s].to_nodes): 
        if (not visited[self.nodes[s].to_nodes[i]]):  
          stack.append(self.nodes[s].to_nodes[i])
          # res.append([s, stack[-1]]) 
          res.append(s) ##
        i += 1

  def DFS(self):
    res = []
    visited = [False] * len(self.nodes)
    for i in range(len(self.nodes)): 
      if (not visited[i]): 
        self.DFSUtil(i, visited, res)
    return res 

  def DFSUtil(self, v, visited, res): 
    visited[v]= True
    res.append(v)
    for i in self.nodes[v].to_nodes: 
        if visited[i] == False: 
            self.DFSUtil(i, visited, res) 

  def DFS(self): 
    res = []
    V = len(self.nodes) 
    visited =[False]*(V) 

    for i in range(V): 
        if visited[i] == False: 
            self.DFSUtil(i, visited, res) 
    return res

  def DFSUtilUndirected (self, v, visited, res):
    visited[v] = True
    neighbors = self.nodes[v].to_nodes + self.nodes[v].from_nodes
    for i in neighbors:
      if visited[i] == False:
        res.append(v)
        self.DFSUtil(i, visited, res)

  def DFSUndirected (self, v):
    res = []
    visited = [False] * len (self.nodes)
    self.DFSUtilUndirected(v, visited, res)
    return res

  def DFSUtilvCode (self, v, visited, res):
    visited[v] = True
    for i in self.nodes[v].to_nodes:
      if visited[i] == False:
        res.append((v, i, VACANT_LABEL))
        self.DFSUtilvCode(i, visited, res)
    for i in self.nodes[v].from_nodes:
      if visited[i] == False:
        res.append((i, v, VACANT_LABEL))
        self.DFSUtilvCode(i, visited, res)

  def DFSvCode(self, v):
    res = []
    visited = [False] * len (self.nodes)
    self.DFSUtilvCode(v, visited, res)
    return res


  def toDFSMin(self):
    dfsMin = MyDFSCode()
    for i in range(self.getNumEdges()):
      temp, s = rightMostPatExt(dfsMin, self)
      if len(temp) == 0:
        print (dfsMin)
        break
      dfsMin.append(s)
    return dfsMin

In [11]:
class MyCodeGraph():
  def __init__(self, data = None, supp = 0, children = None):
    if data is not None:
      assert isinstance(data, MyDFSCode)
      self.data = data.mcopy()
    else:
      self.data = data
    self.support = supp
    self.children = []
    if children is not None:
      for child in children:
        self.addChild(child)
  
  def addChild(self, child):
    assert isinstance(child, MyCodeGraph)
    self.children.append(child)
  
  def updateSupport(self, supp):
    self.support = supp

  def getLeafCodes(self):
    res = []
    def _get_leaf_nodes(node):
        if node is not None:
            if len(node.children) == 0:
                res.append((node.data.mcopy(), node.support))
            for n in node.children:
                _get_leaf_nodes(n)
    _get_leaf_nodes(self)
    return res
  
  def getAllCodes(self):
    res = []
    def _get_all_nodes(node):
        if node is not None:
          if node.data is not None:
            res.append(node)
          for n in node.children:
            _get_all_nodes(n)
    _get_all_nodes(self)
    return res

  def getAllData(self):
    res = []
    def _get_all_nodes(node):
        if node is not None:
          if node.data is not None:
            res.append((node.data.mcopy(), node.support))
          for n in node.children:
            _get_all_nodes(n)
    _get_all_nodes(self)
    return res

  def getAllDFSCode(self):
    res = []
    def _get_all_dfs(node):
        if node is not None:
          if node.data is not None:
            res.append(node.data.mcopy())
          for n in node.children:
            _get_all_dfs(n)
    _get_all_dfs(self)
    return res

  def getBFSearch(self):
    nodes = []
    queue = [self]
    while queue:
        cur_node = queue[0]
        queue = queue[1:]
        if cur_node.data is not None:
          nodes.append((cur_node.data.mcopy(), cur_node.support))
        for child in cur_node.children:
            queue.append(child)
    return nodes

  def getTrueLeaf(self):
    res = []
    leafs = self.getLeafCodes()
    for i in range(len(leafs)):
      flag = True
      for j in range(len(leafs)):
        if len(leafs[i][0]) < len(leafs[j][0]):
          iso = subgraphIsomorphisms3(leafs[i][0], leafs[j][0].toGraph())
          if len(iso) != 0:
            flag = False
            break
      if flag:
        res.append(leafs[i])
    return res
    
  def getNum(self):
    if not isinstance(self, MyCodeGraph):
      return 0
    count = 1
    for i in self.children:
      count+= i.getNum()
    return count

  def __eq__(self, other):
    return (self.data == other.data and self.support == other.support and self.children == other.children)

  def __ne__(self, other):
    return not self.__eq__(other)

  def __repr__(self):
    return '{} : {}'.format(self.data, self.support)
    
  def display (self):
    print (self)
    for i in self.children:
      i.display()

In [12]:
class MyDFSEdge(tuple):
  def isForward(self):
    return self[0] < self[1]
  
  def isBackward(self):
    return self[0] > self[1]
  
  def eqOrder(self, other):
    return (self[0] == other[0] and self[1] == other[1])


  def ltOrder(self, other):
    if self[0] < self[1]:
      if other[0] < other[1]:
        return (other[0] <= self[1] and other[1] == self[1] + 1)
      else:
        return (other[0] == self[1] and other[1] < self[0])
    else:
      if other[0] < other[1]:
        return (other[0] <= self[0] and other[1] == self[0] + 1)
      else:
        return (other[0] == self[0] and other[1] < self[1])


  # def ltOrder(self, other):
  #   if self[0] < self[1]:
  #     if other[0] < other[1]:
  #       return (self[1] < other[1] or (self[1] == other[1] and self[0] > other[0]))
  #     else:
  #       return (other[0] >= self[1])
  #   else:
  #     if other[0] < other[1]:
  #       return True
  #     else:
  #       return (self[0] < other[0] or (self[0] == other[0] and self[1] < other[1]))
  

  def __lt__(self, other):
     return (self.ltOrder(other) or (self[2] < other[2] and self.eqOrder(other)))

  def __gt__(self, other):
    return not (self.__lt__(other) or self.__eq__(other))

  def __repr__(self):
    return '{} {} {}'.format(self[0], self[1], self[2])

In [13]:
## Version: resp version 3 - Using LIST for MF
## return list of list ordered node_index resp nodes in c - 2D

def subgraphIsomorphisms(c, g):   ## c: dfscode, g: graph
  res = []
  for i in range(len(g.nodes)):
    if g.nodes[i].label == c[0][2][0]:
      res.append([i])

  for i in c:         ## O(|c|)
    i1, i2, (vlb1, elb, vlb2) = i[0], i[1], i[2] 
    temp = []
    for j in res:     ## O(|V(g)|) 
      if i2 >= len(j):
        for xx in g.nodes[j[i1]].to_nodes:   ## O(|V(g|)
          if g.nodes[xx].label == vlb2 and xx not in j:
            j1 = j.copy()
            j1.append(xx)
            temp.append(j1.copy())
      elif i1 >= len(j):
        for xx in g.nodes[j[i2]].from_nodes:
          if g.nodes[xx].label == vlb1 and xx not in j:
            j1 = j.copy()
            j1.append(xx)
            temp.append(j1.copy())
      else:
        if MyEdge((j[i1], j[i2], elb)) in g.edges:
            temp.append(j.copy())
    res = temp.copy()
  return res

In [14]:
## Version 3
## O(|E(g)|.|V(g)|.|V(c)|.|c|) ~ O(|E(g)|.|V(g)|.|c|^2)
def rightMostPatExt(dfscode, g):   
  rmNode = dfscode.getRMNode()  ## rightMost vertex of dfscode    ## O(|c|)
  res = set()   ## set of dfsEdges
  
  if rmNode < 0:   ## dfs is None
    for i in g.edges:    ## O(|E(g)|)
      res.add(MyDFSEdge((0, 1, (g.nodes[i[0]].label, i[2], g.nodes[i[1]].label))))
    temp = min(res)
    return res, temp
  else:
    minExt = [rmNode, rmNode, -1, -1]
    gc = dfscode.toGraph()    ## O(|c|^2)
    R = dfscode.getRMPathNodes() ## list of nodes in rightMostPath of dfscode  ## O(|c|^2)
    iso = subgraphIsomorphisms(dfscode, g)    ## O(|c|.|V(g)|^2)
    for i in iso:   ## i: dict          ## O(|E(g)|)
      ## backward ext
      ## backward edge
      for x in g.nodes[i[rmNode]].to_nodes:  ## x: node_index  ## O(|V(g)|)
        if x in i:             ## O(|V(c)|)
          v = i.index(x)
          if MyEdge((rmNode, v, VACANT_LABEL)) not in gc.edges and v in R:   ## O(|c|)
            b = MyDFSEdge((rmNode, v, (gc.nodes[rmNode].label, VACANT_LABEL, gc.nodes[v].label)))
            res.add(b)
            if minExt[0] > v:  #find min
              minExt[0] = v
      ## forward edge
      for x in g.nodes[i[rmNode]].from_nodes:
        if x in i:
          v = i.index(x)
          if MyEdge((v, rmNode, VACANT_LABEL)) not in gc.edges and v in R:   ## O(|c|)
            b = MyDFSEdge((v, rmNode, (gc.nodes[v].label, VACANT_LABEL, gc.nodes[rmNode].label)))
            res.add(b)
            if minExt[1] > v:
              minExt[1] = v
      ## forward ext
      for u in R:      ## O(|R|) ~ O(|V(c)|)
        ## forward edge
        for x in g.nodes[i[u]].to_nodes: ## O(|V(g)|)
          if x not in i:        ## O(|V(c)|) 
            f = MyDFSEdge((u, rmNode + 1, (gc.nodes[u].label, VACANT_LABEL, g.nodes[x].label)))
            res.add(f)
            if minExt[2] < u:   ## find max
              minExt[2] = u
        ## backward edge
        for x in g.nodes[i[u]].from_nodes:
          if x not in i:
            f = MyDFSEdge((rmNode + 1, u, (g.nodes[x].label, VACANT_LABEL, gc.nodes[u].label)))
            res.add(f)
            if minExt[3] < u:
              minExt[3] = u
  
  if min(minExt[0], minExt[1]) != rmNode:
    if minExt[0] <= minExt[1]:
      temp = MyDFSEdge((rmNode, minExt[0], (VACANT_INDEX, VACANT_LABEL, VACANT_INDEX)))
    else:
      temp = MyDFSEdge((minExt[1], rmNode,(VACANT_INDEX, VACANT_LABEL, VACANT_INDEX)))
  else:
    if minExt[2] >= minExt[3]:
      temp = MyDFSEdge((minExt[2], rmNode+1,(VACANT_INDEX, VACANT_LABEL, VACANT_INDEX)))
    else:
      temp = MyDFSEdge((rmNode + 1, minExt[3], (VACANT_INDEX, VACANT_LABEL, VACANT_INDEX)))
  
  for i in res:
    if temp[0] == i[0] and temp[1] == i[1]:
      if temp[2][0] == -1 or temp[2] > i[2]:
        temp = i

  return res, temp

In [15]:
class MyDFSCode(list):
  #def __init__(self):
    # self.rmPath = list() ##list index of vertex from rightmost vertex to root in DFSCode 

  def __eq__(self, other):
    la, lb = len(self), len(other)
    if la != lb:
        return False
    for i in range(la):
        if self[i] != other[i]:
            return False
    return True

  def __ne__(self, other):
    return not self.__eq__(other)

  def getRMNode(self):
    res = -1
    for i in self:
      res = max(res, i[0], i[1])
    return res
  
  def toRMPath(self):
    ''' list of edge_index from right_most_vertex to root'''
    rmPath = []
    old = None
    for i in range(len(self)-1, -1, -1):
      if i == len(self) - 1:
        rmPath.append(i)
      temp = self[i]
      i1, i2 = temp[0], temp[1]
      if i1 < i2 and (old == None or old == i2):
        rmPath.append(i)
        old = i1
    return rmPath

  # def getRMPathNodes(self):
  #   res = []
  #   for i in self.toRMPath():
  #     i1, i2 = self[i][0], self[i][1]
  #     if i1 not in res:
  #       res.append(i1)
  #     if i2 not in res:
  #       res.append(i2)
  #   res = sorted(res, reverse =  True)
  #   return res
  
  def getRMPathNodes(self):
    g = self.toGraph()
    end = self.getRMNode()
    res = find_shortest_path(g, 0, end, [])
    if not res:
      res = find_shortest_path_2(g, 0, end, [])
    res = list(set(res))
    return sorted(res, reverse = True)

  
  def toGraph(self):
    g = MyGraph()
    nodes = []
    nodes_map = dict()
    for i in self:   ## O(|c|)
      i1, i2, (v1, e, v2) = i[0], i[1], i[2]
      if i1 not in nodes:   ## O(|V(c)|)
        nodes.append(i1)
        nodes_map[i1] = len(nodes) - 1
        g.addNode(MyNode(nodes_map[i1], v1))
      if i2 not in nodes:
        nodes.append(i2)
        nodes_map[i2] = len(nodes) - 1
        g.addNode(MyNode(nodes_map[i2], v2))
      g.addEdge(MyEdge((nodes_map[i1], nodes_map[i2], e)))
    return g

  def isMin(self):
    g = self.toGraph()
    dfsMin = MyDFSCode()
    for i in range(len(self)):
      temp, s = rightMostPatExt(dfsMin, g)
      # print (temp, s)
      # print (dfsMin)
      if len(temp) == 0:
        print ('isMin', dfsMin, self)
      if s != self[i]:
        return False
      dfsMin.append(s)
    return True

  
  def extDFSCodeFromIndex(self, index):
    res = []
    g = self.toGraph()
    extDFS = g.DFSvCode(index)
    for i in g.edges:
      if i not in extDFS:
        extDFS.append(i)
    for i in extDFS:
      res.append(g.getDFSEdge(i))
    return res

  def mcopy(self):
    res = type(self)()
    for i in self:
      res.append(i)
    return res
  
  def isDFSCode(self):
    for i in range(len(self)-1):
      if self[i] >= self[i+1]:
        return False
    return True

# FUNCTIONS

In [16]:
def find_shortest_path(graph, start, end, path=[]):
  path = path + [start]
  if start == end:
      return path
  temp = graph.nodes[start].to_nodes.copy()
  if len(temp) == 0 :
      return None
  shortest = None
  for node in temp:
      if node not in path:
          newpath = find_shortest_path(graph, node, end, path)
          if newpath:
              if not shortest or len(newpath) < len(shortest):
                  shortest = newpath
  return shortest

In [17]:
def find_shortest_path_2(graph, start, end, path=[]):
  path = path + [start]
  if start == end:
      return path
  temp = graph.nodes[start].to_nodes + graph.nodes[start].from_nodes
  if len(temp) == 0 :
      return None
  shortest = None
  for node in temp:
      if node not in path:
          newpath = find_shortest_path_2(graph, node, end, path)
          if newpath:
              if not shortest or len(newpath) < len(shortest):
                  shortest = newpath
  return shortest

# drop_vertex_under_support Function

In [18]:
def drop_vertex_under_support(G, s):
  res = MyGraph()
  count_support_dict = G.getLabelNum() # count each label
  
  #drop vertex
  idx_map = {} #{idx_old: idx_new}  -  Memory: [V]*2*sizeof(int)[+epsilon]
  idx_vertex = 0
  for node in G.nodes: # O(|V|)
    if count_support_dict.get(node.label, -1) >= s:
      res.nodes.append(MyNode(idx_vertex, node.label))
      idx_map[node.id] =  idx_vertex
      idx_vertex += 1

  #drop edge
  for edge in G.edges: # O(|E|)
    if idx_map.get(edge[0], -1) != -1 and idx_map.get(edge[1], -1) != -1:
      res.edges.add(MyEdge((idx_map[edge[0]], idx_map[edge[1]], edge[2])))
      res.nodes[idx_map[edge[0]]].to_nodes.append(idx_map[edge[1]])
      res.nodes[idx_map[edge[1]]].from_nodes.append(idx_map[edge[0]])

  return res

# Improve FPMiner

In [19]:
## return list of dfscode generate from c
def genCandidateOpt(c, lbMatchingTo, lbMatchingFrom): 
  res = set()
  R = c.getRMPathNodes()
  rmNode = c.getRMNode()
  gc = c.toGraph()
  ## backward ext
  for i in R[1:]:
    if MyEdge((rmNode, i, VACANT_LABEL)) not in gc.edges:  ## be
      res.add(MyDFSEdge((rmNode, i, (gc.nodes[rmNode].label, VACANT_LABEL, gc.nodes[i].label))))
    if MyEdge((i, rmNode, VACANT_LABEL)) not in gc.edges:  ## fe
      res.add(MyDFSEdge((i, rmNode, (gc.nodes[i].label, VACANT_LABEL, gc.nodes[rmNode].label))))
  ## forward ext
  for u in R:
    for xx in lbMatchingTo[gc.nodes[u].label]:
      res.add(MyDFSEdge((u, rmNode + 1, (gc.nodes[u].label, VACANT_LABEL, xx))))
    for xx in lbMatchingFrom[gc.nodes[u].label]:
      res.add(MyDFSEdge((rmNode + 1,u, (xx, VACANT_LABEL, gc.nodes[u].label))))
  
  return res


In [20]:
def convertToGrami(f1, f2, f):
  MODIFY_INDEX = 0
  graph = MyGraph()
  graph.loadData(f1, f2)
  with open(f, 'w') as fi:
    fi.write('# t 1\n')
    for n in graph.nodes:
      fi.write(n.__repr__()+'\n')
    for e in graph.edges:
      fi.write(e.__repr__()+'\n')

In [21]:
def convertMFtoMS2(MF):
  if len(MF) == 0:
    return list()
  MS = []
  for i in range(len(MF[0])):
    MS.append(dict())
  for i in MF:
    for key in i:
      MS[key][i[key]] = 1 if i[key] not in MS[key] else MS[key][i[key]]+1
  return MS 

In [22]:
def readOutputGraMi(file):
  cont = []
  with open(file, 'r') as f:
    for line in f.readlines():
      cont.append(line.strip('\n'))
  ## get num of dfs codes
  num = int(cont[1])

  ## get list of graphs
  res = []
  for i in range(num):
    res.append([])
  count = -1
  for i in cont[2:]:
    if i[-1] == ':':
      count += 1
      continue
    res[count].append(i)

  resDFS = []

  for i in res:
    dfs = MyDFSCode()
    nodes = []
    for j in i:
      temp = j.split(' ')
      if temp[0] == 'v':
        nodes.append(int(temp[2]))
      else:
        dfs.append(MyDFSEdge((int(temp[1]), int(temp[2]), (nodes[int(temp[1])], VACANT_LABEL, nodes[int(temp[2])]))))
    resDFS.append(dfs.mcopy()) 

  return resDFS

In [23]:
def compareGraMi(outGraMi, outFPM):
  res = []
  outDFS = outFPM.getAllDFSCode()
  for dfs in outDFS:
    if dfs not in outGraMi:
      res.append(dfs.mcopy())
  for dfs in outGraMi:
    if dfs not in outDFS:
      res.append(dfs.mcopy())
  return res

In [24]:
## Version: resp version 3 - Using LIST for MF
## return list of list ordered node_index resp nodes in c - 2D

def subgraphIsomorphisms3(c, g):   ## c: dfscode, g: graph
  res = []
  for i in range(len(g.nodes)):
    if g.nodes[i].label == c[0][2][0]:
      res.append([i])
  for i in c:         ## O(|c|)
    i1, i2, (vlb1, elb, vlb2) = i[0], i[1], i[2] 
    temp = []
    for j in res:     ## O(|V(g)|) 
      if i2 > i1: ## forward edge
        if i2 >= len(j):
          for xx in g.nodes[j[i1]].to_nodes:   ## O(|V(g|)
            if g.nodes[xx].label == vlb2 and xx not in j:
                ##and getLabel(xx, u) == elb:## ignore label edge
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
        else:
          if j[i1] in g.nodes[j[i2]].from_nodes:
            temp.append(j.copy())
      else:  ## backward edge
        if i1 < len(j):
          if j[i2] in g.nodes[j[i1]].to_nodes:
            temp.append(j.copy())
        else:
          for xx in g.nodes[j[i2]].from_nodes:
            if g.nodes[xx].label == vlb1 and xx not in j:
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
    res = temp.copy()
  return res

In [25]:
## Cacultating support from LIST 2D of MF
def support2D(img):
  if len(img) == 0:
    return 0
  temp = []
  for i in range(len(img[0])):
    temp.append(set())
  for i in range (len(img)):
    for j in range(len(img[i])):
      temp[j].add(img[i][j])
  res = len(temp[0])
  for i in temp:
    if res > len(i):
      res = len(i)
  return res

In [26]:
## O(|E|^2)
## Version resp 4
def MinerPattern2Opt4(g, s):
  QSet = dict()  ##  {vevlb: DFSCode resp}
  QMF = dict() ## {vevlb: [list of list: image of DFSCode]} all
  supps = dict()
  nodes = set()
  MF = dict () ## {vevlb: [list of list]} : frequent patterns
  for e in g.edges:
    dfse = g.getDFSEdge(e)
    if dfse[2] not in QMF:
      QMF[dfse[2]] = [[e[0], e[1]]]
    else:
      QMF[dfse[2]].append([e[0], e[1]])
  for key in QMF:
    sup = support2D(QMF[key])
    if sup >= s:
      QSet[key] = MyDFSCode([MyDFSEdge((0, 1, key))])
      supps[key] = sup
      nodes.update(v[0] for v in QMF[key])
      nodes.update(v[1] for v in QMF[key])
      MF[key] = QMF[key].copy()
  nodes = list(nodes)
  return QSet, supps, MF, nodes

In [27]:
def pruneEdge4(g, MF, nodes):
  newGraph = MyGraph()
  nodes_map = dict()
  for i in range(len(nodes)):
    nodes_map[nodes[i]] = i
    newGraph.nodes.append(MyNode(i, g.nodes[nodes[i]].label))
  for i in MF:
    index = 0
    for v in MF[i]:
      v0, v1 = nodes_map[v[0]], nodes_map[v[1]] 
      newGraph.edges.add(MyEdge((v0, v1, VACANT_LABEL)))
      newGraph.nodes[v0].to_nodes.append(v1)
      newGraph.nodes[v1].from_nodes.append(v0)
      MF[i][index] = [v0, v1].copy()
      index += 1 
  return newGraph

In [28]:
def convertMFtoMS6(img):
  if len(img) == 0:
    return 0
  temp = []
  for i in range(len(img[0])):
    temp.append(set())
  for i in range (len(img)):
    for j in range(len(img[i])):
      temp[j].add(img[i][j])
  return temp

In [29]:
## Version: resp version 7
## Key Idea: Recursive
## Target: Optimize time
## return SUPPORT & MS

def subgraphIsomorphisms7(c, g, s, MS):   ## c: dfscode, g: graph
  node0 = MS[0].copy()

  count = 0

  res = []
  for i in range(c.getRMNode() + 1):
    res.append(set())
  
  for n0 in node0:
    iso = []
    iso.append([n0])
    for i in c[:-1]:         ## O(|c|)
      i1, i2, (vlb1, elb, vlb2) = i[0], i[1], i[2] 
      temp = []
      for j in iso:     ## O(|V(g)|) 
        if i2 >= len(j):  ## fw ext
          for xx in g.nodes[j[i1]].to_nodes:   ## O(|V(g|)
            if xx in MS[i2] and xx not in j:
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
        elif i1 >= len(j):  ## fw ext bw ed
          for xx in g.nodes[j[i2]].from_nodes:
            if xx in MS[i1] and xx not in j:
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
        else:
          if MyEdge((i1, i2, elb)) in g.edges:
            temp.append(j.copy())
    
      iso = temp.copy()
    
    i1, i2, (vlb1, elb, vlb2) = c[-1][0], c[-1][1], c[-1][2] 
    for j in iso:     ## O(|V(g)|) 
      if i2 >= len(j):  ## fw ext
        for xx in g.nodes[j[i1]].to_nodes:   ## O(|V(g|)
          if g.nodes[xx].label == vlb2 and xx not in j:
            j1 = j.copy()
            j1.append(xx)
            temp.append(j1.copy())
      elif i1 >= len(j):  ## fw ext bw ed
        for xx in g.nodes[j[i2]].from_nodes:
          if g.nodes[xx].label == vlb1 and xx not in j:
            j1 = j.copy()
            j1.append(xx)
            temp.append(j1.copy())
      else:
        if MyEdge((i1, i2, elb)) in g.edges:
          temp.append(j.copy())
    
    iso = temp.copy()

    if len(iso) == 0:
      count += 1
      if count > len(node0) - s:
        return -1, list()
    else:
      for k in iso:
        for l in range(len(k)):
          res[l].add(k[l])
  
  supp = len(res[0])
  for i in res:
    if supp > len(i):
      supp = len(i)

  return supp, res

In [30]:
## version 8 resp - using subgraphIso8

def PatExtOpt7(c, g, s, temp, lbTo, lbFrom, MS):
  print ('start gen')
  cdds = genCandidateOpt(c, lbTo, lbFrom)
  print ('end gen', len(cdds))
  print ('c', c)
  for cd in cdds:
    new = c.mcopy()
    new.append(cd)
    if new.isMin():
      print ('cd', cd)
      sup, newMS = subgraphIsomorphisms7(new, g, s, MS)
      print ('sup', sup)
      if sup >= s:
        newtemp = MyCodeGraph(new, sup)
        temp.addChild(newtemp)
        print ('newtemp', newtemp)
        PatExtOpt7(new, g, s, newtemp, lbTo, lbFrom, newMS)
      else:
        newMS.clear()

In [31]:
def FPMinerOpt7(g, s):
  g1 = drop_vertex_under_support(g, s)
  QSet, s2, MF2, nodes = MinerPattern2Opt4(g1, s)
  g2 = pruneEdge4(g1, MF2, nodes)
  lbTo = g2.getLabelMatching()
  lbFrom = g2.getLabelMatchingFrom()
  res = MyCodeGraph()
  print (len(g2.edges), len(g2.nodes))
  for i in QSet:
    temp = MyCodeGraph(QSet[i], s2[i])
    res.addChild(temp)
    MS = convertMFtoMS6(MF2[i])
    # index = 0 if len(MS[0]) <= len(MS[1]) else 1
    PatExtOpt7(QSet[i], g2, s, temp, lbTo, lbFrom, MS)
  return res

# Improve 8

In [32]:
## Version: resp version 8
## Improve from version 6 - reuse MS - extension from node with min supp
## Target: Optimize time
## return SUPPORT & MS

def subgraphIsomorphisms8(c, g, s, MS, index):   ## c: dfscode, g: graph
  node0 = MS[index].copy()

  count = 0

  res = []
  for i in range(c.getRMNode() + 1):
    res.append(set())

  for n0 in node0:
    aa = [-1] * len(res)
    aa[index] = n0
    iso = []
    iso.append(aa.copy())
    for i in c.extDFSCodeFromIndex(index):         ## O(|c|)
      i1, i2, (vlb1, elb, vlb2) = i[0], i[1], i[2] 
      temp = []
      for j in iso:     ## O(|V(g)|) 
        if j[i2] == -1:  ## fw ext
          for xx in g.nodes[j[i1]].to_nodes:   ## O(|V(g|)
            if g.nodes[xx].label == vlb2 and xx not in j:
              ##and getLabel(xx, u) == elb:## ignore label edge
              j[i2] = xx
              temp.append(j.copy())
        elif j[i1] == -1:  ## backward edge 
          for xx in g.nodes[j[i2]].from_nodes:
            # if ((i1 < len(MS) and xx in MS[i1]) or g.nodes[xx].label == vlb1) and xx not in j:
            if g.nodes[xx].label == vlb1 and xx not in j:
              j[i1] = xx
              temp.append(j.copy())
        else:
          if MyEdge((j[i1], j[i2], elb)) in g.edges:
            temp.append(j.copy())
      
      iso = temp.copy()
    
    if len(iso) == 0:
      count += 1
      if count > len(node0) - s:
        return -1, list(), -1
    else:
      for k in iso:
        for l in range(len(k)):
          res[l].add(k[l])
  
  for i in range(len(res)):
    if len(res[index]) > len(res[i]):
      index = i

  return len(res[index]), res, index

In [33]:
def recursive(dfse, iso, g):
  newiso = []
  i1, i2, (vlb1, elb, vlb2) = dfse[0], dfse[1], dfse[2]
  for i in iso:
    if i2 >= len(i):
      temp = []
      for xx in g.nodes[i1].to_nodes:
        if g.nodes[xx].label == vlb2 and xx not in j:
          temp.append(xx)
      if len(temp) > 0:
        newiso.append([i, temp])


In [34]:
## version 8 resp - using subgraphIso8

def PatExtOpt8(c, g, s, temp, lbTo, lbFrom, MS, index):
#   print ('start gen')
  cdds = genCandidateOpt(c, lbTo, lbFrom)
#   print ('end gen', len(cdds))
#   print ('c', c)
  for cd in cdds:
    new = c.mcopy()
    new.append(cd)
    if new.isMin():
#       print ('cd', cd)
      sup, newMS, newIndex = subgraphIsomorphisms8(new, g, s, MS, index)
#       print ('sup', sup, newIndex)
      if sup >= s:
        newtemp = MyCodeGraph(new, sup)
        temp.addChild(newtemp)
#         print ('newtemp', newtemp)
        PatExtOpt8(new, g, s, newtemp, lbTo, lbFrom, newMS, newIndex)
      else:
        newMS.clear()

In [35]:
def FPMinerOpt8(g, s):
  g1 = drop_vertex_under_support(g, s)
  QSet, s2, MF2, nodes = MinerPattern2Opt4(g1, s)
  g2 = pruneEdge4(g1, MF2, nodes)
  lbTo = g2.getLabelMatching()
  lbFrom = g2.getLabelMatchingFrom()
  res = MyCodeGraph()
  print (len(g2.edges), len(g2.nodes))
  for i in QSet:
    temp = MyCodeGraph(QSet[i], s2[i])
    res.addChild(temp)
    MS = convertMFtoMS6(MF2[i])
    index = 0 if len(MS[0]) <= len(MS[1]) else 1
    PatExtOpt8(QSet[i], g2, s, temp, lbTo, lbFrom, MS, index)
  return res

# RuleGen

In [36]:
def repRuleGen(gc, n):
  s, conf = [], []
  q = queue.Queue()
  gcList = gc.getBFSearch()   ## get all (data, supp) in codeGraph by BFS
  leafList = gc.getTrueLeaf()
  for i in range(len(leafList)):
    q = queue.Queue()
    q.put(leafList[i])
    while not q.empty():
      qR = q.get()
      ancestors = genAncestors(gcList, qR[0])
      for ql, qr in ancestors:    ## get all ancestors ql of qR
        q.put(ql)
        if qR[1]/ql[1] >= n and isConnected(qr):
          if (ql[0], qr) not in s:
            s.append((ql[0], qr))
            conf.append(qR[1]/ql[1])
  return s, conf

In [37]:
def ruleGen(gc, n):
  s, conf = [], []
  q = queue.Queue()
  gcList = gc.getBFSearch()   ## get all (data, supp) in codeGraph by BFS
  for i in range(len(gcList)):
    q = queue.Queue()
    q.put(gcList[i])
    while not q.empty():
      qR = q.get()
      ancestors = genAncestors(gcList, qR[0])
      for ql, qr in ancestors:    ## get all ancestors ql of qR
        q.put(ql)
        if qR[1]/ql[1] >= n and isConnected(qr):
          if (ql[0], qr) not in s:
            s.append((ql[0], qr))
            conf.append(qR[1]/ql[1])
  return s, conf

In [38]:
def genAncestors(gc, dfscode):
  res = []
  for i in gc:
    if len(i[0]) < len(dfscode):
      iso = subgraphIsomorphisms(i[0], dfscode.toGraph())
      if len(iso) > 0:
        for j in iso:
          temp = MyDFSCode()
          for k in range(len(i[0])):
            dfse = MyDFSEdge((j[i[0][k][0]], j[i[0][k][1]], i[0][k][2]))
            temp.append(dfse)
          res.append((temp.mcopy(), i[1]))  
    else:
      break
  return res

In [39]:
def genAncestors(gc, dfscode):
  res = []
  for i in gc:
    if len(i[0]) < len(dfscode):
      iso = subgraphIsomorphisms(i[0], dfscode.toGraph())
      if len(iso) > 0:
        nodes_map = iso[0].copy()
        dfs = MyDFSCode()
        for k in i[0]:
            dfse = MyDFSEdge((nodes_map[k[0]], nodes_map[k[1]], k[2]))
            dfs.append(dfse)
        qr = genConsequence(dfs, dfscode)
        
        newqr = MyDFSCode()
        for k in qr:
            if k[0] not in nodes_map:
              nodes_map.append(k[0])
            if k[1] not in nodes_map:
              nodes_map.append(k[1])
            newqr.append(MyDFSEdge((nodes_map.index(k[0]), nodes_map.index(k[1]), k[2])))
        
        res.append(((i[0].mcopy(), i[1]), newqr)) 
    else:
      break
  return res

In [40]:
def genAncestors(gc, dfscode):
  res = []
  for i in gc:
    if len(i[0]) < len(dfscode):
      iso = subgraphIsomorphisms(i[0], dfscode.toGraph())
      if len(iso) > 0:
        nodes_map = iso[0].copy()
        dfs = MyDFSCode()
        for k in i[0]:
            dfse = MyDFSEdge((nodes_map[k[0]], nodes_map[k[1]], k[2]))
            dfs.append(dfse)
        qr = genConsequence(dfs, dfscode)
        
        newqr = MyDFSCode()
        for k in qr:
            if k[0] not in nodes_map:
              nodes_map.append(k[0])
            if k[1] not in nodes_map:
              nodes_map.append(k[1])
            newqr.append(MyDFSEdge((nodes_map.index(k[0]), nodes_map.index(k[1]), k[2])))
            
        res.append(((i[0].mcopy(), i[1]), newqr)) 
    else:
      break
  return res

In [41]:
def isConnected(dfscode):
  g = dfscode.toGraph()
  # g.display()
  dfs = g.DFSUndirected(0)
  # print (dfs)
  return len(dfs) == len(g.nodes) 

In [42]:
def genConsequence(anc, code):
  res = MyDFSCode()
  for i in code:
    if i not in anc:
      res.append(i)
  return res

In [43]:
def toMinRule(ql, qr):
  g = MyGraph()
  nodes = []
  nodes_map = dict()
  for i in ql:   ## O(|c|)
    i1, i2, (v1, e, v2) = i[0], i[1], i[2]
    if i1 not in nodes_map:   ## O(|V(c)|)
      nodes.append(i1)
      nodes_map[i1] = len(nodes) - 1
      g.addNode(MyNode(nodes_map[i1], v1))
    if i2 not in nodes_map:
      nodes.append(i2)
      nodes_map[i2] = len(nodes) - 1
      g.addNode(MyNode(nodes_map[i2], v2))
    g.addEdge(MyEdge((nodes_map[i1], nodes_map[i2], e)))
  newql = g.toDFSMin()
  newqr = MyDFSCode()
  for i in qr:
    if i[0] not in nodes_map:
      nodes.append(i[0])
      nodes_map[i[0]] = len(nodes) - 1
    if i[1] not in nodes_map:
      nodes.append(i[1])
      nodes_map[i[1]] = len(nodes) - 1
    newqr.append(MyDFSEdge((nodes_map[i[0]], nodes_map[i[1]], i[2])))
  return newql, newqr

In [44]:
def genNewEdges(rule, g):
    res = set()
    labelDict = g.getLabelDict()

    ql, qr = rule[0], rule[1]
    iso = subgraphIsomorphisms(ql, g)
    for c in qr:
      i1, i2, (vlb1, elb, vlb2) = c[0], c[1], c[2] 
      temp = []
      for j in iso:     ## O(|V(g)|) 
        if i2 >= len(j):
          for xx in labelDict[vlb2]:   ## O(|V(g|)
            if MyEdge((j[i1], xx, elb)) not in g.edges and xx not in j:
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
              res.add(MyEdge((j[i1], xx, elb)))
        elif i1 >= len(j):
          for xx in labelDict[vlb1]:
            if MyEdge((xx, j[i2], elb)) not in g.edges and xx not in j:
              j1 = j.copy()
              j1.append(xx)
              temp.append(j1.copy())
              res.add(MyEdge((xx, j[i2], elb)))
        else:
          if MyEdge((j[i1], j[i2], elb)) not in g.edges:
            temp.append(j.copy())
            res.add(MyEdge((j[i1], j[i2], elb)))
      iso = temp.copy()
    return res

In [45]:
def genNewEdges(rule, g):
    res = set()
    labelDict = g.getLabelDict()

    ql, qr = rule[0], rule[1]
#     print ('rule', rule)
    iso = subgraphIsomorphisms(ql, g)
    for i in iso:
        isoo = []
        isoo.append(i.copy())
        for c in qr:
#             print ('c', c)
            i1, i2, (vlb1, elb, vlb2) = c[0], c[1], c[2] 
            temp = []
            for j in isoo:     ## O(|V(g)|) 
#                 print ('j' , j)
                if i2 >= len(j) and i1 >= len(j):
                    break
                if i2 >= len(j):
                  for xx in labelDict[vlb2]:   ## O(|V(g|)
                    if MyEdge((j[i1], xx, elb)) not in g.edges and xx not in j:
                        j1 = j.copy()
                        j1.append(xx)
                        temp.append(j1.copy())
                elif i1 >= len(j):
                  for xx in labelDict[vlb1]:
                    if MyEdge((xx, j[i2], elb)) not in g.edges and xx not in j:
                        j1 = j.copy()
                        j1.append(xx)
                        temp.append(j1.copy())
                else:
                  if MyEdge((j[i1], j[i2], elb)) not in g.edges:
                    temp.append(j.copy())
                    
            isoo = temp.copy()
        if (len(isoo) > 0):
            for k in isoo:
                for c in qr:
                    res.add(MyDFSEdge((k[c[0]], k[c[1]], c[2])))
    return res

In [46]:
def plot(self, newEdges):
    try:
      import networkx as nx
      import matplotlib.pyplot as plt
      import matplotlib.colors as mcolors
    except Exception as e:
      print('Can not plot graph: {}'.format(e))
      return
    #cycle = list(mcolors.CSS4_COLORS.keys())
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    gnx = nx.DiGraph()
    vlbs = {}
    elbs = {}
    vids = {}
    for i in range(len(self.nodes)):
      gnx.add_node(i, label= self.nodes[i].label)
      vlbs[i] = self.nodes[i].label
      vids[i] = i
    for e in self.edges:
      gnx.add_edge(e[0], e[1], label=e[2])
      elbs[(e[0], e[1])] = 0
    for e in newEdges:
      gnx.add_edge(e[0], e[1], label = e[2])
      elbs[(e[0], e[1])] = 1
    colors = ['black', 'red']
    vcolors = [cycle[i%len(cycle)] for i in vlbs.values()]
    ecolors = [colors[i] for i in elbs.values()]
    fsize = (min(12, 1 * len(self.nodes)),
              min(12, 1 * len(self.nodes)))
    plt.figure(3, figsize=fsize)
    pos = nx.circular_layout(gnx)
    # pos = nx.spring_layout(gnx)
    nx.draw_networkx(gnx, pos, node_size = 200, node_color = vcolors, edge_color = ecolors, labels = vids)
    #nx.draw_networkx_edge_labels(gnx, pos, edge_labels= elbs)
    plt.autoscale(enable = True)
    plt.show()


In [47]:
def plot(self, newEdges):
    try:
      import networkx as nx
      import matplotlib.pyplot as plt
      import matplotlib.colors as mcolors
    except Exception as e:
      print('Can not plot graph: {}'.format(e))
      return
    #cycle = list(mcolors.CSS4_COLORS.keys())
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    gnx = nx.DiGraph()
    vlbs = {}
    vids = {}
    
    for i in range(len(self.nodes)):
      gnx.add_node(i, label= self.nodes[i].label)
      vlbs[i] = self.nodes[i].label
      vids[i] = i
    for e in self.edges:
        gnx.add_edge(e[0], e[1], color = 'black')
        
    for e in newEdges:
        gnx.add_edge(e[0], e[1], color = 'red')
    
    vcolors = [cycle[i%len(cycle)] for i in vlbs.values()]
    ecolors = [c for (u, v, c) in gnx.edges.data('color')]
    fsize = (min(12, 1 * len(self.nodes)),
              min(12, 1 * len(self.nodes)))
    plt.figure(3, figsize=fsize)
    pos = nx.circular_layout(gnx)
    # pos = nx.spring_layout(gnx)
    nx.draw_networkx(gnx, pos, node_size = 200, node_color = vcolors, edge_color = ecolors, labels = vids)
    #nx.draw_networkx_edge_labels(gnx, pos, edge_labels= elbs)
    plt.autoscale(enable = True)
    plt.show()


In [48]:
def getNMaxrule(confRuleList, n): # return index_list of n_max confs.
    if n == 0:
        return []
    if n == 1:
        return [max(confRuleList)]
    result = [None]*(n+1)
    max_conf = [0.0]*(n+1)
    for i in range(len(confRuleList)):
        j = n
        while j>0 and max_conf[j-1]<confRuleList[i]:
            max_conf[j] = max_conf[j-1]
            max_conf[j-1] = confRuleList[i]
            result[j] = result[j-1]
            result[j-1] = i
            j -= 1
    return result[:-1]

In [184]:
a =[345,46,68,87,89,346457,90,56,436,76808]
b = max(a)
[b,]

[346457]

In [133]:
a =[345,46,68,87,89,90,56,436,76808]
b =[4,657,78,8008,76,78,476]
x = getNMaxrule(a, 1)
res = []
for x_ in x:
    res.append(a[x_])
res

[76808, 436, 345, 90, 89]

In [50]:
a = set(['465','67','78','89','90','0','0'])
a.remove('465')
a

{'0', '67', '78', '89', '90'}

# Split K Fold

In [51]:
def device_data(profiles_filename, relationships_filename, num_partition):
  def device_edges():
    print('Split test data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_partition):
      map_idx[i] = set()
      edges_old.append([])

    # des_filename = relationships_filename.split('.')[0]
    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_partition ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_partition)
        
      lines = set(lines)

      for i in range(0, num_partition-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        # des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          # des_f.write('{}\t{}\n'.format(data[0], data[1]))
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])
        # des_f.close()

      # des_f = open(des_filename+'_'+str(num_partition-1)+'.txt', 'w')
      print('Partition ', num_partition-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        # des_f.write('{}\t{}\n'.format(data[0], data[1]))
        edges_old[num_partition-1].append([data[0], data[1]])
        map_idx[num_partition-1].add(data[0])
        map_idx[num_partition-1].add(data[1])
      # des_f.close()

    print("Done!")
    return map_idx, edges_old

  def modified_vertex_idx(map_idx):
    map_new = {} #{node_index: {file_idx: node_index_modified},....}
    for key in map_idx:
      idx = 0
      for vertex in map_idx[key]:
        if vertex not in map_new:
          map_new[vertex] = {key: idx}
        else:
          map_new[vertex][key] = idx
        idx += 1
    return map_new

  def device_vertex():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]
    
      print('Split edge ........')
      map_idx, edges_old = device_edges() #{file_index: set(vertex_idxs)}
      print('Modify vertex index ............')
      new_map = modified_vertex_idx(map_idx) #{node_index_old: {file_idx: node_index_modified},....}
      # print(map_idx)
      # print(edges_old)
      # print(new_map)

      # Split and update index profile
      print('Update index profiles:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_partition):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Update index edges in relations_file_splited
      print('Update index relationships:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_partition):
        print('Partition ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex()

In [52]:
a = set([243,46,67,78,89,90])
b = random.sample(a, 3)
b

[78, 90, 89]

In [53]:
def device_k_fold(profiles_filename, relationships_filename, num_fold):
  def device_edges_():
    print('Split train data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_fold):
      map_idx[i] = set()
      edges_old.append([])

    # des_filename = relationships_filename.split('.')[0]
    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_fold ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_fold)
    
      lines = set(lines)

      for i in range(0, num_fold-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        # des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          # des_f.write('{}\t{}\n'.format(data[0], data[1]))
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])
        # des_f.close()

      # des_f = open(des_filename+'_'+str(num_partition-1)+'.txt', 'w')
      print('Partition ', num_fold-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        # des_f.write('{}\t{}\n'.format(data[0], data[1]))
        edges_old[num_fold-1].append([data[0], data[1]])
        map_idx[num_fold-1].add(data[0])
        map_idx[num_fold-1].add(data[1])
      # des_f.close()

    print("Done!")
    return map_idx, edges_old

  def split_k_fold(map_idx, edges_old):
    # map_idx: {file_index: set(vertex_idxs)}
    # edges_old #edges[file_index] = [[v0,v1], ....]

    fold_map = {} #{node_index: {fold_idx: node_index_modified},....}
    map_idx_new = {} # map_idx_new: {fold_index: set(vertex_idxs)}
    edges_old_new = []

    for i in range(0, num_fold):
      map_idx_new[i] = set()
      edges_old_new.append([])

    # map_idx_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          map_idx_new[i] = set(map_idx_new[i] | map_idx[j])
    # edges_old_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          edges_old_new[i] = edges_old_new[i] + edges_old[j]

    # map_idx_new: {fold_index: set(vertex_idxs)}
    for fold in map_idx_new:
      idx = 0
      for vertex in map_idx_new[fold]:
        if vertex not in fold_map:
          fold_map[vertex] = {fold: idx}
        else:
          fold_map[vertex][fold] = idx
        idx += 1

#     print(fold_map)
#     print(map_idx_new)
#     print(edges_old_new)
    return fold_map, map_idx_new, edges_old_new

  def device_vertex_():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]

      print('Split edge ..........')
      map_idx, edges_old = device_edges_() #{file_index: set(vertex_idxs)}

      edge = edges_old
      print('Modify vertex index ............')
      new_map, map_idx, edges_old = split_k_fold(map_idx, edges_old) #{node_index_old: {file_idx: node_index_modified},....}

      # Split and update index profile
      print('Update index profiles:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Update index of edges in relationships_file_splited
      print('Update index relationships:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex_()

In [54]:
def device_k_fold_v1(profiles_filename, relationships_filename, num_fold):
  def device_edges_():
    print('Split train data......................')
    map_idx = {} #{file_index: set(vertex_idxs)}
    edges_old = [] #edges[file_index] = [[v0,v1], ....]
    for i in range(0, num_fold):
      map_idx[i] = set()
      edges_old.append([])

    # des_filename = relationships_filename.split('.')[0]
    with open(relationships_filename, 'r') as edges_f:
      print('Data reading ........')
      lines = edges_f.readlines()
      print('Cal max item_per_fold ......')
      max_num_items_per_partition = math.ceil(len(lines)/num_fold)
    
      lines = set(lines)

      for i in range(0, num_fold-1):
        print('Partition ', i, '.........')
        edges = random.sample(lines, max_num_items_per_partition)

        # des_f = open(des_filename+'_'+str(i)+'.txt', 'w')
        for edge in edges:
          lines.remove(edge)
          data = edge.strip('\n').split('\t')
          # des_f.write('{}\t{}\n'.format(data[0], data[1]))
          edges_old[i].append([data[0], data[1]])
          map_idx[i].add(data[0])
          map_idx[i].add(data[1])
        # des_f.close()

      # des_f = open(des_filename+'_'+str(num_partition-1)+'.txt', 'w')
      print('Partition ', num_fold-1, '.........')
      for line in lines:
        data = line.strip('\n').split('\t')
        # des_f.write('{}\t{}\n'.format(data[0], data[1]))
        edges_old[num_fold-1].append([data[0], data[1]])
        map_idx[num_fold-1].add(data[0])
        map_idx[num_fold-1].add(data[1])
      # des_f.close()

    print("Done!")
    return map_idx, edges_old

  def split_k_fold(map_idx, edges_old):
    # map_idx: {file_index: set(vertex_idxs)}
    # edges_old #edges[file_index] = [[v0,v1], ....]

    fold_map = {} #{node_index: {fold_idx: node_index_modified},....}
    map_idx_new = {} # map_idx_new: {fold_index: set(vertex_idxs)}
    edges_old_new = []

    for i in range(0, num_fold):
      map_idx_new[i] = set()
      edges_old_new.append([])

    # map_idx_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          map_idx_new[i] = set(map_idx_new[i] | map_idx[j])
    # edges_old_new
    for i in range(0, num_fold):
      for j in range(0, num_fold):
        if i != j:
          edges_old_new[i] = edges_old_new[i] + edges_old[j]

    # map_idx_new: {fold_index: set(vertex_idxs)}
    for fold in map_idx_new:
      idx = 0
      for vertex in map_idx_new[fold]:
        if vertex not in fold_map:
          fold_map[vertex] = {fold: idx}
        else:
          fold_map[vertex][fold] = idx
        idx += 1

#     print(fold_map)
#     print(map_idx_new)
#     print(edges_old_new)
    return fold_map, map_idx_new, edges_old_new

  def device_vertex_():
    with open(profiles_filename, 'r') as f:
      lines = f.readlines()
      map_vertex_labels = {}
      for line in lines:
        data = line.strip('\n').split('\t')
        map_vertex_labels[data[0]] = data[1]

      print('Split edge ..........')
      map_idx, edges_old = device_edges_() #{file_index: set(vertex_idxs)}

      edge = edges_old
      print('Modify vertex index ............')
      new_map, map_idx, edges_old = split_k_fold(map_idx, edges_old) #{node_index_old: {file_idx: node_index_modified},....}

      # Split and update index profile
      print('Update index profiles:')
      des_filename = profiles_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        vertex_set = set()
        for vertex in map_idx[i]:
          vertex_set.add((new_map[vertex][i], map_vertex_labels[vertex]))
        vertex_set = sorted(vertex_set)
        for vertex in vertex_set:
          des_f.write('{}\t{}\n'.format(vertex[0], vertex[1]))
        des_f.close()

      # Update index of edges in relationships_file_splited
      print('Update index relationships:')
      des_filename = relationships_filename.split('.')[0]
      for i in range(0, num_fold):
        print('Fold ', i, '.........')
        des_f = open(des_filename+'_'+str(i)+'_train.txt', 'w')
        edge_set = set()
        for edge in edges_old[i]:
          # print(edge)
          edge_set.add((new_map[edge[0]][i], new_map[edge[1]][i]))
        edge_set = sorted(edge_set)
        for edge in edge_set:
          des_f.write('{}\t{}\n'.format(edge[0], edge[1]))
        des_f.close()

    print("Done!")

  device_vertex_()

In [55]:
# print('Test: ')
# device_data('kFoldData/test2/test2_profiles.txt', 'kFoldData/test2/test2_relationships.txt', 5)
# print('Train: ')
# device_k_fold('kFoldData/test2/test2_profiles.txt', 'kFoldData/test2/test2_relationships.txt', 5)

# Convert Data GramiFormat -> MyFormat / MyFormat -> GramiFormat

In [ ]:
Grami_source_path = 'GraMi/Datasets/'
Grami_des_path = 'Data_demo/'

# convert data from gramiFormat to myDataFormat
def data_convert_Grami(source_filename, des_path): #des_profile_filename, des_relationships_filename):
  des_profile_filename = source_filename.split('.')[0] + '_profiles.txt' 
  des_relationships_filename = source_filename.split('.')[0] + '_relationships.txt'
  with open(Grami_source_path + source_filename, 'r') as f:
    with open(Grami_des_path + des_profile_filename, 'w') as profiles_f:
      with open(Grami_des_path + des_relationships_filename, 'w') as relationships_f:
        lines = f.readlines()
        for line in lines:
          data = line.split(' ')#.strip('\n').split(' ')
          if data[0] == 'v':
            profiles_f.write('{}\t{}'.format(data[1], data[2]))
          if data[0] == 'e':
            # if data[1]==data[2]:
            #   print('{} {}'.format(data[1], data[2]))
            relationships_f.write('{}\t{}\n'.format(data[1], data[2]))
    print('Converted from {} to {} and {}!'.format(Grami_source_path+source_filename, Grami_des_path+des_profile_filename, Grami_des_path+des_relationships_filename))

In [ ]:
# data_convert_Grami('mico.lg', Grami_des_path)

In [ ]:
source_path = "Data_demo/mico/"
des_path = "GraMi/Datasets/"

# Convert data from mydataFormat to GramiFormat
def data_convert_to_Grami(profiles_filename, relationships_filename):
  pf_ = source_path + profiles_filename
  rf_ = source_path + relationships_filename
  des_filename = des_path + profiles_filename.split('.')[0] + '.lg'
  with open(pf_, 'r') as pf:
    with open(rf_, 'r') as rf:
      with open(des_filename, 'w') as f:
        f.write('t\t#\t1\n')
        profiles = pf.readlines()
        # print(profiles)
        for profile in profiles:
          data = profile.strip('\n').split('\t')
          f.write('v {} {}\n'.format(data[0], data[1]))
        print('Profiles! Done!')
        relationships = rf.readlines()
        # print(relationships)
        for relationship in relationships:
          data = relationship.strip('\n').split('\t')
          # print(data)
          f.write('e {} {} -1\n'.format(data[0], data[1]))
        print('Relationships! Done!')
  print('Done!')

In [ ]:
# data_convert_to_Grami('mico_profiles_4_train.txt', 'mico_relationships_4_train.txt')

# Measure Acc

In [56]:
def get_top_pair_node(G, measure_matrix, n):
  set_matrix = set()
  for i in range(len(measure_matrix)):
    for j in range(len(measure_matrix)):
      if measure_matrix[i][j] != 0.0 and measure_matrix[i][j]  != 1.0:
        set_matrix.add((measure_matrix[i][j], i, j))
        # set_matrix.add((i, j))
        
    result = set_matrix.copy()
    for edge in G.edges:
      v1, v2, l = edge
      for ed in set_matrix:
        w_, v1_, v2_ = ed
        if v1==v1_ and v2==v2_:
          result.remove(ed)
  result = sorted(result, reverse=True)
  return set(list(result)[:n])

def Jaccard_measure(node1, node2):
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  return len(set(node1.to_nodes).intersection(set(node2.to_nodes))) / len(list(set(node1.to_nodes) | set(node2.to_nodes)))

def cal_jaccard_measure_matrix(G):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Jaccard_measure(node1, node2)
  return result_matrix

def Jaccard_measure_v1(node1, node2):
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  return len(set(node1.to_nodes+node1.from_nodes).intersection(set(node2.to_nodes+node2.from_nodes))) / len(list(set(node1.to_nodes+node1.from_nodes) | set(node2.to_nodes+node2.from_nodes)))

def cal_jaccard_measure_matrix_v1(G):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Jaccard_measure_v1(node1, node2)
  return result_matrix

def Simrank_measure(G, node1, node2, C):
  if node1.id == node2.id:
    return 1
  if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
    return 0
  sum_sim = 0
  for node1_ in G.nodes[node1.id].to_nodes:
    for node2_ in G.nodes[node2.id].to_nodes:
      sum_sim += Simrank_measure(G, G.nodes[node1_], G.nodes[node2_], C)
  return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix(G, C):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  for node1 in G.nodes:
    for node2 in G.nodes:
      result_matrix[node1.id][node2.id] = Simrank_measure(G, node1, node2, C)
  return result_matrix



In [111]:
def get_predict_all_pair_node(G, measure_matrix):
  set_matrix = set()
  for i in range(len(measure_matrix)):
    for j in range(len(measure_matrix)):
      if measure_matrix[i][j] != 0.0 and measure_matrix[i][j]  != 1.0:
        set_matrix.add((measure_matrix[i][j], i, j))
        
    result = set_matrix.copy()
    for edge in G.edges:
      v1, v2, l = edge
      for ed in set_matrix:
        w_, v1_, v2_ = ed
        if v1==v1_ and v2==v2_:
          result.remove(ed)
  result = sorted(result, reverse=True)
  return set(list(result))

def Simrank_measure(G, node1, node2, C, iter_, max_iter):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes:
        for node2_ in G.nodes[node2.id].to_nodes:
            if iter_ <= max_iter:        
                sum_sim += Simrank_measure(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix(G, C, max_iter):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  count = 0
  for node1 in G.nodes:
    for node2 in G.nodes:
      count += 1
      if count%1000==0:
            print(count)
      result_matrix[node1.id][node2.id] = Simrank_measure(G, node1, node2, C, 0, max_iter)
  return result_matrix

# def Simrank_measure_v1(G, node1, node2, C, iter_, max_iter, set_edge_checked):
#     if node1.id == node2.id:
#         return 1
#     if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
#         return 0
#     sum_sim = 0
#     for node1_ in G.nodes[node1.id].to_nodes+G.nodes[node1.id].from_nodes:
#         for node2_ in G.nodes[node2.id].to_nodes+G.nodes[node2.id].from_nodes:
#             if iter_ <= max_iter:
#                 if (node1_, node2_) not in set_edge_checked:
# #                     print('call ', node1_, node2_)
#                     set_edge_checked.add((node1_, node2_))
# #                     print('Len(cheked) = ', len(set_edge_checked))
#                     if len(set_edge_checked) < 1000: # Upper threshold for edge_checked
#                         sum_sim += Simrank_measure_v1(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter, set_edge_checked)
#                     else:
# #                         print('Break edge_checked!')
#                         C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))
#     return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def Simrank_measure_v1(G, node1, node2, C, iter_, max_iter, set_edge_checked):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes+G.nodes[node1.id].from_nodes:
        for node2_ in G.nodes[node2.id].to_nodes+G.nodes[node2.id].from_nodes:
            if iter_ <= max_iter:
                if (node1_, node2_) not in set_edge_checked:
                    set_edge_checked.add((node1_, node2_))
                    sum_sim += Simrank_measure_v1(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter, set_edge_checked)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix_v1(G, C, max_iter):
#   print('Init result matrix.................')
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
#   count = 0
#   print('start: ')
  for node1 in G.nodes:
#     count += 1
#     if count%1000==0:
#       print(count)
    for node2 in G.nodes:
#       print('edge: ', count, node1.id, node2.id, '.......................')
      result_matrix[node1.id][node2.id] = Simrank_measure_v1(G, node1, node2, C, 0, max_iter, set())
  return result_matrix

In [58]:
def countEdgeGenInPredictedEdges(edgesGen, edgesPredicted):
    # edgesGen: gen from rule. {(v1, v2, vlv),....}
    # edgesPredicted: SimRank/Jaccard top-L predicted [(similarity, v1, v2),...]
    set_predicted = set()
    for edge in edgesPredicted:
        _, v1, v2 = edge
        set_predicted.add((v1, v2))
        
    result = 0
    for edge in edgesGen:
        v1, v2, vlv = edge
        if (v1, v2) in set_predicted:
            result += 1
    return result

In [179]:
def testPipeline(dataName, supp, cnf, minTopL, maxTopL, stepTopL, KFold, NTopRules):
    result = {'jaccard': {}, 'simrank': {}}
#     for i in range(0, KFold):
#         result['jaccard'][i] = {}
#         result['simrank'][i] = {}
    for i in range(0, KFold):
#         if i == 0:
#             continue
        print('----------------- Fold ', i, '--------------------')
        test = MyGraph()
        train = MyGraph()
        test.loadData(dataName+'_profiles_'+str(i)+'.txt', dataName+'_relationships_'+str(i)+'.txt')
        train.loadData(dataName+'_profiles_'+str(i)+'_train.txt', dataName+'_relationships_'+str(i)+'_train.txt')
        
        print('Minning FP.......')
        gc = FPMinerOpt8(train, supp)
        print('Gen Rules........')
        rules, confs = ruleGen(gc, cnf)

        if len(rules)==0:
            print('Not rule!')
            continue
        top_rule_idx = getNMaxrule(confs, NTopRules)
#         top_rules = []
#         for i in top_rule_idx:
#             top_rules.append([rules[i], confs[i]])
        
#         ttt = genNewEdges(rules[top_rule_idx[0]], test) # Rule 0
    
        ttt = set()
#         print('top rule idxs: ', top_rule_idx)
#         print('Rules: ', rules)
        for rule_idx in top_rule_idx:
#             print(rules[rule_idx])
            if rule_idx == None:
                continue
            ttt = ttt.union(genNewEdges(rules[rule_idx], test))
#             print(rules[top_rule_idx[rule_idx]])
#             ttt = ttt.union(genNewEdges(rules[top_rule_idx[rule_idx]], test))
        
        print('Cal simrank_matrix........')
        simrank_matrix = cal_simrank_measure_matrix_v1(test, 0.8, 5)
        predict_simrank = get_predict_all_pair_node(test, simrank_matrix)
        predict_simrank = sorted(predict_simrank)

        print('Cal jaccard_matrix........')
        jaccard_matrix = cal_jaccard_measure_matrix_v1(test)
        predict_jaccard = get_predict_all_pair_node(test, jaccard_matrix)
        predict_jaccard = sorted(predict_jaccard)
        
        topL = minTopL
        while topL <= maxTopL:
            if topL not in result['jaccard']:
                result['jaccard'][topL] = {}
                result['simrank'][topL] = {}
            print('TopL = ', topL)
            Lrs = countEdgeGenInPredictedEdges(ttt, predict_simrank[-topL:])
            Lrj = countEdgeGenInPredictedEdges(ttt, predict_jaccard[-topL:])
            print('Jaccard: ', Lrj/topL, ' SimRank: ', Lrs/topL)
            result['jaccard'][topL][i] = Lrj/topL
            result['simrank'][topL][i] = Lrs/topL
            topL += stepTopL
    
    topL = minTopL
    while topL <= maxTopL:
        sum_sim = 0.0
        sum_jac = 0.0
        for i in range(0, KFold):
            sum_jac += result['jaccard'][topL][i]
            sum_sim += result['simrank'][topL][i]
        result['jaccard'][topL]['Mean'] = sum_jac/KFold
        result['simrank'][topL]['Mean'] = sum_sim/KFold
        topL += stepTopL
    
    with open(dataName+'_result_acc_top_'+str(NTopRules)+'rule.txt', 'w') as f:
        json.dump(result, f)
    
    return result

In [185]:
# testPipeline('kFoldData/citeseer/citeseer', 100, 0.9, 100, 500, 50, 5, 5)
data_pool = ['kFoldData/citeseer/citeseer', 'kFoldData/cora/cora', 'kFoldData/mico/mico', 'kFoldData/test1/test1', 'kFoldData/test2/test2'] 
sup_pool = [100, 150, 10500, 260, 90]
conf_pool = [0.9, 0.9, 0.9, 0.9, 0.9]
minTopL = [50, 50, 50, 50, 50]
maxTopL = [500, 500, 500, 500, 500]
stepTopL = [50, 50, 50, 50, 50]
kFold = [5, 5, 5, 5, 5]
nTopRule = [5, 5, 5, 5, 5]

select = 1
result = testPipeline(data_pool[select], sup_pool[select], conf_pool[select], minTopL[select], maxTopL[select], stepTopL[select], kFold[select], nTopRule[select])

----------------- Fold  0 --------------------
Minning FP.......
2650 1784
Gen Rules........
Cal simrank_matrix........
Cal jaccard_matrix........
TopL =  50
Jaccard:  0.1  SimRank:  0.22
TopL =  100
Jaccard:  0.08  SimRank:  0.18
TopL =  150
Jaccard:  0.07333333333333333  SimRank:  0.14666666666666667
TopL =  200
Jaccard:  0.1  SimRank:  0.125
TopL =  250
Jaccard:  0.092  SimRank:  0.112
TopL =  300
Jaccard:  0.08  SimRank:  0.12333333333333334
TopL =  350
Jaccard:  0.11142857142857143  SimRank:  0.13142857142857142
TopL =  400
Jaccard:  0.12  SimRank:  0.115
TopL =  450
Jaccard:  0.14  SimRank:  0.11333333333333333
TopL =  500
Jaccard:  0.144  SimRank:  0.108
----------------- Fold  1 --------------------
Minning FP.......
3013 2086
Gen Rules........
Cal simrank_matrix........
Cal jaccard_matrix........
TopL =  50
Jaccard:  0.1  SimRank:  0.42
TopL =  100
Jaccard:  0.07  SimRank:  0.32
TopL =  150
Jaccard:  0.14666666666666667  SimRank:  0.23333333333333334
TopL =  200
Jaccard:  0.14

# Measure Acc(R)

#  dddd

In [163]:
# test
MODIFY_INDEX = 0
example = MyGraph()
example.loadData('kFoldData/citeseer/citeseer_profiles_0_train.txt', 'kFoldData/citeseer/citeseer_relationships_0_train.txt')
# example.plot()
# test
# MODIFY_INDEX = 0
# test = MyGraph()
# example.loadData('kFoldData/citeseer/citeseer_profiles_0.txt', 'kFoldData/citeseer/citeseer_relationships_0.txt')
# example.plot()

In [164]:
fc = FPMinerOpt8(example, 100)
fc.display()
rules, confs = ruleGen(fc, 0.9)
# top5 = getNMaxrule(confs, 5)
# top5
rules

2646 2316
None : 0
[0 1 (1, -1, 1)] : 273
[0 1 (1, -1, 1), 0 2 (1, -1, 1)] : 122
[0 1 (1, -1, 1), 1 2 (1, -1, 1)] : 115
[0 1 (1, -1, 1), 1 2 (1, -1, 1), 3 2 (1, -1, 1)] : 105
[0 1 (1, -1, 1), 1 2 (1, -1, 1), 3 2 (1, -1, 1), 3 4 (1, -1, 1)] : 101
[0 1 (1, -1, 1), 2 1 (1, -1, 1)] : 181
[0 1 (1, -1, 1), 2 1 (1, -1, 1), 2 3 (1, -1, 1)] : 116
[0 1 (1, -1, 1), 2 1 (1, -1, 1), 2 3 (1, -1, 1), 4 1 (1, -1, 1)] : 102
[0 1 (1, -1, 1), 2 1 (1, -1, 1), 2 3 (1, -1, 1), 0 4 (1, -1, 1)] : 110
[0 1 (1, -1, 1), 2 1 (1, -1, 1), 3 1 (1, -1, 1)] : 107
[0 1 (4, -1, 4)] : 211
[0 1 (2, -1, 2)] : 299
[0 1 (2, -1, 2), 2 1 (2, -1, 2)] : 122
[0 1 (2, -1, 2), 0 2 (2, -1, 2)] : 119
[0 1 (5, -1, 5)] : 222
[0 1 (0, -1, 0)] : 252
[0 1 (0, -1, 0), 0 2 (0, -1, 0)] : 105
[0 1 (0, -1, 0), 1 2 (0, -1, 0)] : 103
[0 1 (0, -1, 0), 2 1 (0, -1, 0)] : 134


[([0 1 (1, -1, 1), 1 2 (1, -1, 1)], [3 2 (1, -1, 1)]),
 ([0 1 (1, -1, 1), 0 2 (1, -1, 1)], [3 1 (1, -1, 1)]),
 ([0 1 (1, -1, 1), 1 2 (1, -1, 1), 3 2 (1, -1, 1)], [3 4 (1, -1, 1)]),
 ([0 1 (1, -1, 1), 2 1 (1, -1, 1), 3 1 (1, -1, 1)], [2 4 (1, -1, 1)]),
 ([0 1 (1, -1, 1), 2 1 (1, -1, 1), 2 3 (1, -1, 1)], [0 4 (1, -1, 1)])]

In [108]:
sim = cal_simrank_measure_matrix_v1(example, 0.8, 5)

Init result matrix.................
start: 
edge:  1 0 0 .......................
edge:  1 0 1 .......................
edge:  1 0 2 .......................
edge:  1 0 3 .......................
edge:  1 0 4 .......................
edge:  1 0 5 .......................
edge:  1 0 6 .......................
edge:  1 0 7 .......................
edge:  1 0 8 .......................
edge:  1 0 9 .......................
edge:  1 0 10 .......................
edge:  1 0 11 .......................
edge:  1 0 12 .......................
edge:  1 0 13 .......................
edge:  1 0 14 .......................
edge:  1 0 15 .......................
edge:  1 0 16 .......................
edge:  1 0 17 .......................
edge:  1 0 18 .......................
edge:  1 0 19 .......................
edge:  1 0 20 .......................
edge:  1 0 21 .......................
edge:  1 0 22 .......................
edge:  1 0 23 .......................
edge:  1 0 24 .......................
edge:  1 0 25 ..

edge:  1 0 213 .......................
edge:  1 0 214 .......................
edge:  1 0 215 .......................
edge:  1 0 216 .......................
edge:  1 0 217 .......................
edge:  1 0 218 .......................
edge:  1 0 219 .......................
edge:  1 0 220 .......................
edge:  1 0 221 .......................
edge:  1 0 222 .......................
edge:  1 0 223 .......................
edge:  1 0 224 .......................
edge:  1 0 225 .......................
edge:  1 0 226 .......................
edge:  1 0 227 .......................
edge:  1 0 228 .......................
edge:  1 0 229 .......................
edge:  1 0 230 .......................
edge:  1 0 231 .......................
edge:  1 0 232 .......................
edge:  1 0 233 .......................
edge:  1 0 234 .......................
edge:  1 0 235 .......................
edge:  1 0 236 .......................
edge:  1 0 237 .......................
edge:  1 0 238 ..........

edge:  1 0 434 .......................
edge:  1 0 435 .......................
edge:  1 0 436 .......................
edge:  1 0 437 .......................
edge:  1 0 438 .......................
edge:  1 0 439 .......................
edge:  1 0 440 .......................
edge:  1 0 441 .......................
edge:  1 0 442 .......................
edge:  1 0 443 .......................
edge:  1 0 444 .......................
edge:  1 0 445 .......................
edge:  1 0 446 .......................
edge:  1 0 447 .......................
edge:  1 0 448 .......................
edge:  1 0 449 .......................
edge:  1 0 450 .......................
edge:  1 0 451 .......................
edge:  1 0 452 .......................
edge:  1 0 453 .......................
edge:  1 0 454 .......................
edge:  1 0 455 .......................
edge:  1 0 456 .......................
edge:  1 0 457 .......................
edge:  1 0 458 .......................
edge:  1 0 459 ..........

edge:  1 0 669 .......................
edge:  1 0 670 .......................
edge:  1 0 671 .......................
edge:  1 0 672 .......................
edge:  1 0 673 .......................
edge:  1 0 674 .......................
edge:  1 0 675 .......................
edge:  1 0 676 .......................
edge:  1 0 677 .......................
edge:  1 0 678 .......................
edge:  1 0 679 .......................
edge:  1 0 680 .......................
edge:  1 0 681 .......................
edge:  1 0 682 .......................
edge:  1 0 683 .......................
edge:  1 0 684 .......................
edge:  1 0 685 .......................
edge:  1 0 686 .......................
edge:  1 0 687 .......................
edge:  1 0 688 .......................
edge:  1 0 689 .......................
edge:  1 0 690 .......................
edge:  1 0 691 .......................
edge:  1 0 692 .......................
edge:  1 0 693 .......................
edge:  1 0 694 ..........

edge:  1 0 902 .......................
edge:  1 0 903 .......................
edge:  1 0 904 .......................
edge:  1 0 905 .......................
edge:  1 0 906 .......................
edge:  1 0 907 .......................
edge:  1 0 908 .......................
edge:  1 0 909 .......................
edge:  1 0 910 .......................
edge:  1 0 911 .......................
edge:  1 0 912 .......................
edge:  1 0 913 .......................
edge:  1 0 914 .......................
edge:  1 0 915 .......................
edge:  1 0 916 .......................
edge:  1 0 917 .......................
edge:  1 0 918 .......................
edge:  1 0 919 .......................
edge:  1 0 920 .......................
edge:  1 0 921 .......................
edge:  1 0 922 .......................
edge:  1 0 923 .......................
edge:  1 0 924 .......................
edge:  1 0 925 .......................
edge:  1 0 926 .......................
edge:  1 0 927 ..........

...

edge:  7 6 668 .......................
edge:  7 6 669 .......................
edge:  7 6 670 .......................
edge:  7 6 671 .......................
edge:  7 6 672 .......................
edge:  7 6 673 .......................
edge:  7 6 674 .......................
edge:  7 6 675 .......................
edge:  7 6 676 .......................
edge:  7 6 677 .......................
edge:  7 6 678 .......................
edge:  7 6 679 .......................
edge:  7 6 680 .......................
edge:  7 6 681 .......................
edge:  7 6 682 .......................
edge:  7 6 683 .......................
edge:  7 6 684 .......................
edge:  7 6 685 .......................
edge:  7 6 686 .......................
edge:  7 6 687 .......................
edge:  7 6 688 .......................
edge:  7 6 689 .......................
edge:  7 6 690 .......................
edge:  7 6 691 .......................
edge:  7 6 692 .......................
edge:  7 6 693 ..........

edge:  7 6 887 .......................
edge:  7 6 888 .......................
edge:  7 6 889 .......................
edge:  7 6 890 .......................
edge:  7 6 891 .......................
edge:  7 6 892 .......................
edge:  7 6 893 .......................
edge:  7 6 894 .......................
edge:  7 6 895 .......................
edge:  7 6 896 .......................
edge:  7 6 897 .......................
edge:  7 6 898 .......................
edge:  7 6 899 .......................
edge:  7 6 900 .......................
edge:  7 6 901 .......................
edge:  7 6 902 .......................
edge:  7 6 903 .......................
edge:  7 6 904 .......................
edge:  7 6 905 .......................
edge:  7 6 906 .......................
edge:  7 6 907 .......................
edge:  7 6 908 .......................
edge:  7 6 909 .......................
edge:  7 6 910 .......................
edge:  7 6 911 .......................
edge:  7 6 912 ..........

edge:  7 6 1120 .......................
edge:  7 6 1121 .......................
edge:  7 6 1122 .......................
edge:  7 6 1123 .......................
edge:  7 6 1124 .......................
edge:  7 6 1125 .......................
edge:  7 6 1126 .......................
edge:  7 6 1127 .......................
edge:  7 6 1128 .......................
edge:  7 6 1129 .......................
edge:  7 6 1130 .......................
edge:  7 6 1131 .......................
edge:  7 6 1132 .......................
edge:  7 6 1133 .......................
edge:  7 6 1134 .......................
edge:  7 6 1135 .......................
edge:  7 6 1136 .......................
edge:  7 6 1137 .......................
edge:  7 6 1138 .......................
edge:  7 6 1139 .......................
edge:  7 6 1140 .......................
edge:  7 6 1141 .......................
edge:  7 6 1142 .......................
edge:  7 6 1143 .......................
edge:  7 6 1144 .......................


edge:  7 6 1357 .......................
edge:  7 6 1358 .......................
edge:  7 6 1359 .......................
edge:  7 6 1360 .......................
edge:  7 6 1361 .......................
edge:  7 6 1362 .......................
edge:  7 6 1363 .......................
edge:  7 6 1364 .......................
edge:  7 6 1365 .......................
edge:  7 6 1366 .......................
edge:  7 6 1367 .......................
edge:  7 6 1368 .......................
edge:  7 6 1369 .......................
edge:  7 6 1370 .......................
edge:  7 6 1371 .......................
edge:  7 6 1372 .......................
edge:  7 6 1373 .......................
edge:  7 6 1374 .......................
edge:  7 6 1375 .......................
edge:  7 6 1376 .......................
edge:  7 6 1377 .......................
edge:  7 6 1378 .......................
edge:  7 6 1379 .......................
edge:  7 6 1380 .......................
edge:  7 6 1381 .......................


KeyboardInterrupt: 

In [76]:
start = time.time()
fc = FPMinerOpt8(example, 90)
print(time.time()-start)
fc.display()

20990 2000
8.217993021011353
None : 0
[0 1 (2, -1, 13)] : 95
[0 1 (13, -1, 14)] : 92
[0 1 (7, -1, 5)] : 102
[0 1 (12, -1, 9)] : 91
[0 1 (5, -1, 11)] : 97
[0 1 (12, -1, 14)] : 100
[0 1 (12, -1, 14), 2 1 (14, -1, 14)] : 94
[0 1 (11, -1, 3)] : 99
[0 1 (6, -1, 3)] : 93
[0 1 (7, -1, 7)] : 110
[0 1 (7, -1, 7), 1 2 (7, -1, 7)] : 92
[0 1 (7, -1, 7), 2 1 (11, -1, 7)] : 90
[0 1 (7, -1, 7), 2 1 (15, -1, 7)] : 93
[0 1 (7, -1, 7), 2 1 (12, -1, 7)] : 90
[0 1 (7, -1, 7), 1 2 (7, -1, 11)] : 92
[0 1 (7, -1, 7), 2 1 (14, -1, 7)] : 95
[0 1 (1, -1, 1)] : 93
[0 1 (11, -1, 12)] : 98
[0 1 (11, -1, 12), 2 1 (14, -1, 12)] : 90
[0 1 (15, -1, 2)] : 97
[0 1 (7, -1, 1)] : 103
[0 1 (1, -1, 12)] : 95
[0 1 (3, -1, 9)] : 96
[0 1 (15, -1, 3)] : 98
[0 1 (14, -1, 15)] : 104
[0 1 (3, -1, 1)] : 97
[0 1 (11, -1, 15)] : 94
[0 1 (3, -1, 15)] : 92
[0 1 (14, -1, 10)] : 96
[0 1 (2, -1, 2)] : 96
[0 1 (7, -1, 3)] : 109
[0 1 (7, -1, 3), 2 0 (7, -1, 7)] : 90
[0 1 (7, -1, 3), 0 2 (7, -1, 7)] : 93
[0 1 (13, -1, 7)] : 91
[0 1 (7, -1, 1

In [77]:
rules, conf = ruleGen(fc, 0.9)
rules

[([0 1 (12, -1, 14)], [2 1 (14, -1, 14)]),
 ([0 1 (14, -1, 14)], [2 1 (12, -1, 14)]),
 ([0 1 (11, -1, 7)], [2 1 (7, -1, 7)]),
 ([0 1 (15, -1, 7)], [2 1 (7, -1, 7)]),
 ([0 1 (12, -1, 7)], [2 1 (7, -1, 7)]),
 ([0 1 (14, -1, 7)], [2 1 (7, -1, 7)]),
 ([0 1 (11, -1, 12)], [2 1 (14, -1, 12)]),
 ([0 1 (14, -1, 12)], [2 1 (11, -1, 12)]),
 ([0 1 (1, -1, 11)], [2 1 (7, -1, 11)]),
 ([0 1 (14, -1, 11)], [2 1 (7, -1, 11)]),
 ([0 1 (14, -1, 12)], [2 1 (3, -1, 12)]),
 ([0 1 (7, -1, 12)], [2 1 (3, -1, 12)]),
 ([0 1 (11, -1, 12)], [2 1 (3, -1, 12)]),
 ([0 1 (7, -1, 12)], [2 1 (14, -1, 12)]),
 ([0 1 (14, -1, 12)], [2 1 (7, -1, 12)]),
 ([0 1 (2, -1, 11)], [2 1 (7, -1, 11)]),
 ([0 1 (2, -1, 11)], [2 1 (3, -1, 11)]),
 ([0 1 (2, -1, 3)], [2 1 (3, -1, 3)]),
 ([0 1 (11, -1, 3)], [2 1 (2, -1, 3)]),
 ([0 1 (2, -1, 3)], [2 1 (11, -1, 3)]),
 ([0 1 (11, -1, 14)], [2 1 (14, -1, 14)]),
 ([0 1 (7, -1, 11)], [2 1 (3, -1, 11)]),
 ([0 1 (3, -1, 11)], [2 1 (7, -1, 11)]),
 ([0 1 (3, -1, 7)], [2 1 (7, -1, 7)]),
 ([0 1 (1, 

In [200]:
edges = genNewEdges(rules[0], test)
len(edges)

KeyboardInterrupt: 

In [ ]:
# jaccard_matrix = cal_jaccard_measure_matrix(example)
# top_jaccard = get_top_pair_node(example, jaccard_matrix, 20)
# top_jaccard

In [ ]:
# ## Check number of edge in top_edges is exist in example 
# top_edges_jaccard_not_in_graph = top_jaccard.copy()
# count = 0
# for edge in example.edges:
#     v1, v2, l = edge
#     for ed in top_jaccard:
#         w, v1_, v2_ = ed
# #         print(edge, ed)
#         if v1==v1_ and v2==v2_:
#             count += 1
#             top_edges_jaccard_not_in_graph.remove(ed)
# print(count, len(top_edges_jaccard_not_in_graph))

# def get_top_new_edges(G, )

In [ ]:
def get_predict_all_pair_node(G, measure_matrix):
  set_matrix = set()
  for i in range(len(measure_matrix)):
    for j in range(len(measure_matrix)):
      if measure_matrix[i][j] != 0.0 and measure_matrix[i][j]  != 1.0:
        set_matrix.add((jaccard_matrix[i][j], i, j))
        
    result = set_matrix.copy()
    for edge in G.edges:
      v1, v2, l = edge
      for ed in set_matrix:
        w_, v1_, v2_ = ed
        if v1==v1_ and v2==v2_:
          result.remove(ed)
  result = sorted(result, reverse=True)
  return set(list(result))

def Simrank_measure(G, node1, node2, C, iter_, max_iter):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes:
        for node2_ in G.nodes[node2.id].to_nodes:
            if iter_ <= max_iter:        
                sum_sim += Simrank_measure(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix(G, C, max_iter):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
  count = 0
  for node1 in G.nodes:
    for node2 in G.nodes:
      count += 1
      if count%1000==0:
            print(count)
      result_matrix[node1.id][node2.id] = Simrank_measure(G, node1, node2, C, 0, max_iter)
  return result_matrix

def Simrank_measure_v1(G, node1, node2, C, iter_, max_iter, set_edge_checked):
    if node1.id == node2.id:
        return 1
    if len(node1.to_nodes)==0 or len(node2.to_nodes)==0:
        return 0
    sum_sim = 0
    for node1_ in G.nodes[node1.id].to_nodes+G.nodes[node1.id].from_nodes:
        for node2_ in G.nodes[node2.id].to_nodes+G.nodes[node2.id].from_nodes:
            if iter_ <= max_iter:
                if (node1_, node2_) not in set_edge_checked:
                    set_edge_checked.add((node1_, node2_))
                    sum_sim += Simrank_measure_v1(G, G.nodes[node1_], G.nodes[node2_], C, iter_+1, max_iter, set_edge_checked)
    return C*(sum_sim/(len(node1.to_nodes)*len(node2.to_nodes)))

def cal_simrank_measure_matrix_v1(G, C, max_iter):
  result_matrix = [[0.0 for x in range(len(G.nodes))] for y in range(len(G.nodes))]
#   count = 0
  for node1 in G.nodes:
    for node2 in G.nodes:
#       count += 1
#       if count%1000==0:
#             print(count)
      result_matrix[node1.id][node2.id] = Simrank_measure_v1(G, node1, node2, C, 0, max_iter, set())
  return result_matrix

# mm = Simrank_measure(example, example.nodes[0], example.nodes[2], 0.8, 0, 5)
# print(mm)

In [ ]:
# ss = Simrank_measure(test, test.nodes[512], test.nodes[3], 0.8, 0, 5)
# ss

In [ ]:
fc = FPMinerOpt8(train, 100)
rules, confs = ruleGen(fc, 0.9)

In [ ]:
top_rule_idx = getNMaxrule(confs, 5)
top_rules = []
for i in top_rule_idx:
    top_rules.append([rules[i], confs[i]])
top_rules

In [ ]:
# start = time.time()
# res = set()
# for i in rgTest2:
#     ttt = genNewEdges(i, test2)
#     res.update(ttt)
# print(time.time()-start)


ttt = genNewEdges(top_rules[0][0], test)

In [ ]:
len(ttt)

In [ ]:
res = set()
for i in ttt:
    v1, v2, vlv = i
    if (v1, v2)not in res:
        res.add((v1, v2))
len(res)

In [ ]:
simrank_matrix = cal_simrank_measure_matrix_v1(test, 0.8, 5)
predict_simrank = get_predict_all_pair_node(test, simrank_matrix)
predict_simrank = sorted(predict_simrank)
predict_simrank

In [ ]:
Lrs = countEdgeGenInPredictedEdges(ttt, predict_simrank[-300:])
Lrs / 300

In [ ]:
jaccard_matrix = cal_jaccard_measure_matrix_v1(test)
predict_jaccard = get_predict_all_pair_node(test, jaccard_matrix)
predict_jaccard = sorted(predict_jaccard)
list(predict_jaccard)[-100:]

In [ ]:
Lrj = countEdgeGenInPredictedEdges(ttt, predict_jaccard[-300:])
Lrj / 300

In [ ]:
# train
MODIFY_INDEX = 0
train = MyGraph()
train.loadData('kFoldData/citeseer/citeseer_profiles_0_train.txt', 'kFoldData/citeseer/citeseer_relationships_0_train.txt')

# test
MODIFY_INDEX = 0
test = MyGraph()
test.loadData('kFoldData/citeseer/citeseer_profiles_0.txt', 'kFoldData/citeseer/citeseer_relationships_0.txt')

In [ ]:
# fp = FPMinerOpt8(train, 100)
# rules = ruleGen(fp, 0.6)

# processes = []
# for i in range(0,4):
#     p = multiprocessing.Process(target=FPMinerOpt8, args=(train, 100))
#     processes.append(p)
#     p.start()

# for process in processes:
#     process.join()

In [ ]:
rules

In [ ]:
jaccard_matrix = cal_jaccard_measure_matrix(test)
top_jaccard = get_top_pair_node(test, jaccard_matrix, 50)
# top_jaccard

In [ ]:
simrank_matrix = cal_simrank_measure_matrix(test, 0.8, 5)
top_simrank = get_top_pair_node(test, simrank_matrix, 50)
# top_simrank

In [ ]:
edges_predicted = genNewEdges(rules, test)

# TEST LOAD_DATA





In [ ]:
MODIFY_INDEX = 0
mygen210test = MyGraph()
mygen210test.loadData('DataGeneration/profiles_v210_test.txt', 'DataGeneration/relationships_v210_test.txt')
# mygen2000.plot()

In [ ]:
MODIFY_INDEX = 0
mygen2000 = MyGraph()
mygen2000.loadData('DataGeneration/profiles_v2000_e100000.txt', 'DataGeneration/relationships_v2000_e100000.txt')
# mygen2000.plot()

In [ ]:
MODIFY_INDEX = 0
mygen500 = MyGraph()
mygen500.loadData("DataGeneration/profiles_v500_e25000.txt", "DataGeneration/relationships_v500_e25000.txt")
# mygen500.plot()

In [ ]:
MODIFY_INDEX = 0
cora = MyGraph()
cora.loadData("RealDataset/cora_ok/profiles.txt", "RealDataset/cora_ok/relationships.txt")
# mygen500.plot()

In [ ]:
MODIFY_INDEX = 0
example = MyGraph()
example.loadData('Data_demo/example_profiles.txt', 'Data_demo/example_relationships.txt')
example.plot()

In [ ]:
MODIFY_INDEX = 0
mico = MyGraph()
mico.loadData('Data_demo/mico_profiles.txt', 'Data_demo/mico_relationships.txt')
## mico.plot()

In [ ]:
MODIFY_INDEX = 0
test1 = MyGraph()
test1.loadData('Data_demo/test1_profiles.txt', 'Data_demo/test1_relationships.txt')
test1.display()
test1.plot()

In [ ]:
MODIFY_INDEX = 0
citeseer = MyGraph()
citeseer.loadData('Data_demo/citeseer_profiles.txt', 'Data_demo/citeseer_relationships.txt')
# citeseer.plot()

In [ ]:
MODIFY_INDEX = 0
citeseer0 = MyGraph()
citeseer0.loadData('Data_demo/citeseer_profiles_0.txt', 'Data_demo/citeseer_relationships_0.txt')
# citeseer.plot()

In [ ]:
MODIFY_INDEX = 0
gr = MyGraph()
gr.loadData('Data_demo/gr_profile.txt', 'Data_demo/gr_relationship.txt')
gr.plot()

In [ ]:
MODIFY_INDEX = 0
demo = MyGraph()
demo.loadData('Data_demo/profile.txt', 'Data_demo/relationship.txt')
demo.plot()

In [ ]:
MODIFY_INDEX = 0
test2 = MyGraph()
test2.loadData('Data_demo/test2_profiles.txt', 'Data_demo/test2_relationships.txt')
test2.plot()

# TEST GRAMI

In [ ]:
% cd GraMi
! chmod +x grami

In [ ]:
! ./grami -f cora.lg -s 150 -t 1 -p 1

In [ ]:
cat Output.txt

In [ ]:
! ./grami -f example.lg -s 2 -t 1 -p 1

In [ ]:
! cat Data_demo/example_profiles.txt

# TEST


In [ ]:
plot(example, ttt)

In [ ]:
example.nodes

In [ ]:
for i in resEx.getAllDFSCode():
  if (len(i)) == 7:
    print(i)
  if len(i) == 5:
    print (i)

In [ ]:
temp  = []
for i in rrgEx:
  if len(i[0]) == 5 and len(i[1]) == 2:
    temp.append(i)

In [ ]:
temp

In [ ]:
if MyEdge((11, 13, -1)) in ttt:
  print('aa')

In [ ]:
ttt = genNewEdges(temp, example)

In [ ]:
ttt

In [ ]:
example.edges

In [ ]:
for i in ttt:
  if i in example.edges:
    print (i)

In [ ]:
len(ttt)

In [ ]:
resDemo.display()

In [ ]:
for i in rrgDemo:
  if len(i[0]) + len(i[1]) > 6:
    print (i)

In [ ]:
rrgTest1= repRuleGen(resTest1, 0.6)

In [ ]:
a0.plot()

In [ ]:
dft

In [ ]:
for i in a:
  if not i.isMin():
    print ('i', i)
    print('min', i.toGraph().toDFSMin())

In [ ]:
len(a)

In [ ]:
compareGraMi(readOutputGraMi('Output.txt'), FPMinerOpt8(cora, 150))

In [ ]:
resTest2 = FPMinerOpt8(test2, 2)

In [ ]:
res8 = FPMinerOpt8(citeseer, 120)

In [ ]:
resDemo.display()

In [ ]:
resDemo = FPMinerOpt8(demo, 2)

In [ ]:
ruleGen(resCiteseer7, 0)

In [ ]:
resCiteseer7 = FPMinerOpt7(citeseer, 120)

In [ ]:
resMico8 = FPMinerOpt8(mico, 12500)

In [ ]:
resCiteseer8 = FPMinerOpt8(citeseer, 120)

In [ ]:
resDemo = FPMinerOpt8(demo, 2)

In [ ]:
rep = repRuleGen(resEx, 1)

In [ ]:
r = ruleGen(resEx, 1)

In [ ]:
temppp = []
count = 0
for i in r:
  if i not in temppp:
    temppp.append(i)
  else:
    count += 1

In [ ]:
dic = [0] * 100
for i in resEx.getAllCodes():
  dic[len(i.data)] += 1

In [ ]:
dic

In [ ]:
for i in a:
  if len(i[0]) == 5 and len(i[1]) == 2 and i not in b:
    print (i)

In [ ]:
b = ruleGen(resEx, 1)

In [ ]:
resEx = FPMinerOpt8(example, 2)

In [ ]:
repRuleGen(res8, 0.9)

In [ ]:
res8 = FPMinerOpt8(citeseer, 120)